In [4]:
import mysql.connector
import json
import pandas as pd
from os import path
import sys

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

class DBMS:
    def __init__(self):
        self.DB = 'PV_DB'
        self.table = 'MedDRA'
        self.user = 'yeschan'
        self.pw = 'yeschan119'
        self.host = 'db-2.cjfiturksrlr.ap-northeast-2.rds.amazonaws.com'
    '''
    ➣ connect to the MySQL with user_name, pw
    ➣ create DB with DB_name
    ➣ return DB_name
    '''
    def create_DB(self):
        try:
            #establishing the connection
            user_name = self.user
            pw = self.pw
            # connect to DBMS(MySQL)
            sql_connector = mysql.connector.connect(
            user = self.user,
            password = self.pw,
            host = self.host)

            cursor = sql_connector.cursor();
            # create DB
            cursor.execute("DROP database if exists " + self.DB)
            cursor.execute("CREATE DATABASE " + self.DB)
            print(self.DB + " is created")
        except mysql.connector.Error as error:
            print("Failed creating DB to MySQL table {}".format(error))

    def create_table(self):
        try:
            DB_connector = mysql.connector.connect(host=self.host,
                                                 database=self.DB,
                                                 user=self.user,
                                                 password=self.pw)
            cursor = DB_connector.cursor()
            #connect to DB
            cursor.execute("DROP TABLE if exists " + self.table)
            #write query to create table
            create_query = '''
            (ARRN INT NOT NULL, SEQ INT, LLT varchar(100), Code varchar(20), 
            Level varchar(10), PT varchar(100), PT_Code varchar(20), 
            HLGT_Code varchar(20), SOC varchar(200), SOC_Code varchar(20), 
            Primary_SOC varchar(5));
            '''
            #create_query = "(ID INT NOT NULL Primary key,"
            #create_query += "Word varchar(30) NOT NULL,"
            #create_query += "Frequency INT NOT NULL);"
            #create table with attributes already defined
            cursor.execute("Create table " + self.table + create_query)
            print(self.table + " table is created...")
        except mysql.connector.Error as error:
            print("Failed creating table to MySQL table {}".format(error))

    def insert_data_to_DB(self, insert_data):
        try:
            sql_connector = mysql.connector.connect(host=self.host,
                                                 database=self.DB,
                                                 user=self.user,
                                                 password=self.pw)
            cursor = sql_connector.cursor()

            '''
            ➣ read data row by row
            ➣ query to insert data to the DB.table
            ➣ insert into DB.table values (%s, %s, %s, %s, %s, %s)
            '''
            for i, row in insert_data.iterrows():
                sql = "insert into " + self.DB +'.' + self.table +" values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
                cursor.execute(sql, tuple(row))
                sql_connector.commit() # sends a COMMIT statement to the MySQL
            print("Data insertion is finished")
        except mysql.connector.Error as error:
            print("Failed inserting json data into MySQL table {}".format(error))
    def get_fetchall_from_DB(self):
        try:
            connector = mysql.connector.connect(host=self.host,
                                                 database=self.DB,
                                                 user=self.user,
                                                 password=self.pw)
            cursor = connector.cursor()
            print("You're connected to db to get data")
            sql = "select SOC from " + self.table
            cursor.execute(sql)
            result = cursor.fetchall()  # crawling data from DB
            result = pd.DataFrame(result)
            return(result)
        except mysql.connector.Error as error:
            print("Failed fetching trained_words from MySQL table {}".format(error))

trained_words = {}
def get_trained_source():
    global trained_words
    try:
        with open("trained_words.json") as json_file:
                trained_words = json.load(json_file)
        return True
    except:
        print("오류!", "소스파일이 존재하지 않습니다.")
        return False
get_trained_source()

ID = list(range(len(trained_words)))

column = ['Word','Frequency']
insert_data = pd.DataFrame(trained_words.items(), columns = column)
insert_data['ID'] = ID

insert_data = insert_data[['ID','Word','Frequency']]
data = pd.read_excel('MedDRA_eng_kor.xlsx')
DB = DBMS()
#DB.create_DB()
#DB.create_table()
#DB.insert_data_to_DB(data)
#create_Table()
#insert_data_to_DB(insert_data)


result = DB.get_fetchall_from_DB()
print(result)
#dic = result.set_index('word').to_dict()['frequency']

You're connected to db to get data
                                                      0
0     Skin and subcutaneous tissue disorders (피부 및 피...
1     Skin and subcutaneous tissue disorders (피부 및 피...
2     Skin and subcutaneous tissue disorders (피부 및 피...
3     Skin and subcutaneous tissue disorders (피부 및 피...
4             Infections and infestations (감염 및 기생충 감염)
...                                                 ...
1236                       Cardiac disorders (각종 심장 장애)
1237                       Cardiac disorders (각종 심장 장애)
1238                       Cardiac disorders (각종 심장 장애)
1239                       Cardiac disorders (각종 심장 장애)
1240                       Cardiac disorders (각종 심장 장애)

[1241 rows x 1 columns]


In [2]:
data = pd.read_excel('MedDRA_eng_kor.xlsx')
print(len(data.columns))

11
